In [5]:
import torch
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import random

import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, models
from load_dataset import BangladeshDataset, BangladeshDatasetJpegs

import numpy as np
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [6]:
def preprocess(img):
    transform = T.Compose([
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #T.Lambda(lambda x: x[None]),
    ])
    return transform(img)

def deprocess(img, should_rescale=True):
    transform = T.Compose([
        T.Lambda(lambda x: x[0]),
        T.Normalize(mean=[0, 0, 0], std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
        T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
        #T.Lambda(rescale) if should_rescale else T.Lambda(lambda x: x),
        T.ToPILImage(),
    ])
    return transform(img)

def rescale(x):
    low, high = x.min(), x.max()
    x_rescaled = (x - low) / (high - low)
    return x_rescaled

In [92]:
def compute_saliency_maps(X, y, model):
    """
    Compute an expenditure saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Wrap the input tensors in Variables
    X_var = Variable(X, requires_grad=True)
    y_var = Variable(y)
    saliency = None
    ##############################################################################
    #       Implement this function. Perform a forward and backward pass through #
    # the model to compute the gradient of the predicted expenditure with        #
    # respect to each input image. You first want to compute the loss over the   #
    # prediction, and then compute the gradients with a backward pass.           #
    ##############################################################################
    outputs = model(X_var)
    print(outputs)
    preds = outputs.data
    criterion = nn.MSELoss()
    loss = criterion(outputs, labels)
    loss.backward()
    gradients = X_var.grad
    saliency, _ = torch.max(gradients.abs(), 1)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    return saliency.data.squeeze()

In [93]:
def show_saliency_maps(X, y, model):
    # Convert X and y from numpy arrays to Torch Tensors
    #X_tensor = torch.cat([preprocess(Image.fromarray(x)) for x in X], dim=0)
    #y_tensor = torch.FloatTensor(y)
    X_tensor = X.float()
    y_tensor = y.float()

    # Compute saliency maps for images in X
    saliency = compute_saliency_maps(X_tensor, y_tensor, model)

    # Convert the saliency map from Torch Tensor to numpy array and show images
    # and saliency maps together.
    saliency = saliency.numpy()
    N = X.shape[0]
    for i in range(N):
        plt.subplot(2, N, i + 1)
        plt.imshow(X[i])
        plt.axis('off')
        plt.title('Sample #{}'.format(i))
        plt.subplot(2, N, N + i + 1)
        plt.imshow(saliency[i], cmap=plt.cm.hot)
        plt.axis('off')
        plt.gcf().set_size_inches(12, 5)
    plt.show()
    

In [99]:
# Download and load the pretrained SqueezeNet model.
model = torchvision.models.resnet18(pretrained=False)

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features

# 1 since we are only predicting household expenditure
model.fc = nn.Linear(num_ftrs, 1)

model.load_state_dict(torch.load("./epochs_50_Tue_Oct_31_19-12-03_2017_finetune_True.model"))

# We don't want to train the model, so tell PyTorch not to compute gradients
# with respect to model parameters.
for param in model.parameters():
    param.requires_grad = False

AttributeError: 'SqueezeNet' object has no attribute 'fc'

In [98]:
#val_data_dir = '/mnt/mounted_bucket'
#val_bangladesh_csv_path = '/home/echartock03/predicting-poverty/data/bangladesh_2015_valid.csv'

#val_dataset = BangladeshDataset(csv_file=val_bangladesh_csv_path, root_dir=val_data_dir)

data_transform = T.Compose([
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    #T.Lambda(lambda x: x[None]),
])

val_data_dir = '../bangladesh_2015_l8_vis_temp'
val_bangladesh_csv_path = '../bangladesh_2015_temp.csv'

val_dataset = BangladeshDatasetJpegs(csv_file=val_bangladesh_csv_path, root_dir=val_data_dir, transform=data_transform)

dataloder = torch.utils.data.DataLoader(val_dataset, batch_size=4,
                                             shuffle=False, num_workers=1)
dataset_size = len(val_dataset)

In [96]:
# visualize a random batch worth of images
images, labels = next(iter(dataloder))
show_saliency_maps(images, labels, model)

Variable containing:
  4.2379   4.7755   6.8461  ...    6.1573   4.4577   4.2758
  5.7943   3.8631   6.5324  ...    5.9313   3.3270   1.8785
  6.5440   6.4007   6.5355  ...    8.2178   5.4383   3.8180
  5.3949   5.8286   4.7822  ...    6.3951   3.2572   1.6115
[torch.FloatTensor of size 4x1000]



AttributeError: 'torch.DoubleTensor' object has no attribute 'requires_grad'